In [174]:
import pandas as pd
from glob import glob
import numpy as np
from datetime import datetime

In [175]:
exclusion_list = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Exclusion all Customers Apr 2024.csv",dtype = "object",low_memory = False)
exclusion_list.head()

,BANK_SPECIFIC,CUSTOMER_TYPE,TITLE,FIRST_NAME,MIDDLE_NAME,SURNAME,PREVIOUS_NAME,COMPANY_NAME,GENDER,ID_TYPE,...,STATUS_OF_ACCOUNT,EXCLUSION_TYPE,ACCOUNT_BRANCH,OWN_BALANCE_SHARE,AUTH_NEGATIVE_BALANCE,CURRENCY_OF_ACCOUNT,AMOUNT_BAL_IN_ORIG_CURRENCY,EXCHANGE_RATE,ACCOUNT_BALANCE_IN_CEDIS,DEDUCT_OVER_DUE_LOAN
0,252041,Individual,Mrs,LINDA,NaN,BOATENG,NaN,NaN,Female,VOTER.ID,...,DORMANT,NaN,Junction Mall Branch,NaN,NaN,GHS,174.12,1,174.12,NaN
1,252590,Individual,Mr,TEI JOHN,NaN,QUAYNOR,NaN,NaN,Male,VOTER.ID,...,DORMANT,NaN,East Legon Branch,NaN,NaN,GHS,163.16,1,163.16,NaN
2,253192,Individual,Mr,NANA KWAME,NaN,ABBAM,NaN,NaN,Male,DRIVING.LICENSE,...,DORMANT,NaN,Airport City Branch,NaN,NaN,GHS,91,1,91,NaN
3,253192,Individual,Mr,NANA KWAME,NaN,ABBAM,NaN,NaN,Male,DRIVING.LICENSE,...,DORMANT,NaN,Airport City Branch,NaN,NaN,GHS,568.3,1,568.3,NaN
4,253057,Individual,Mr,SAMUEL,NaN,ASARE,NaN,NaN,Male,PASSPORT,...,DORMANT,NaN,Adum Branch,NaN,NaN,GHS,235.13,1,235.13,NaN


In [176]:
exclusion_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130012 entries, 0 to 130011
Data columns (total 35 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   BANK_SPECIFIC                130012 non-null  object
 1   CUSTOMER_TYPE                130012 non-null  object
 2   TITLE                        112627 non-null  object
 3   FIRST_NAME                   112612 non-null  object
 4   MIDDLE_NAME                  0 non-null       object
 5   SURNAME                      112559 non-null  object
 6   PREVIOUS_NAME                0 non-null       object
 7   COMPANY_NAME                 17385 non-null   object
 8   GENDER                       112627 non-null  object
 9   ID_TYPE                      121136 non-null  object
 10  ID_NUMBER                    120369 non-null  object
 11  COMPANY_NUMBER               0 non-null       object
 12  DATE_OF_BIRTH                130003 non-null  object
 13  HOME_ADDRESS  

In [177]:
exclusion_list['DATE_OF_BIRTH'].replace('19000100',np.nan,inplace = True)

In [178]:
LD = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\SANTIZATION FILES\ContractsApr2024.csv",dtype ="object", low_memory = False)
exclusion_list = pd.merge(exclusion_list,LD, left_on = "CUSTOMER_ACCOUNT_NUMBER", right_on = "CURRENT_ACCOUNT_NO", how = "left")
acc_ownership = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\ACC_OWNER_UNIT.csv", dtype = "object", low_memory = False)
exclusion_list = pd.merge(exclusion_list,acc_ownership, left_on = "ACCOUNT_BY_OWNERSHIP", right_on = "ACCOUNT_BY_OWNERSHIP" , how = "left")

In [179]:
exclusion_list = exclusion_list.drop_duplicates(subset = "CUSTOMER_ACCOUNT_NUMBER", keep = "first")
# exclusion_list.info()

## DATE_FORMATTING_FUNCTION

In [180]:
def transform_date(date_str):
    # Convert the original date to a datetime object
    date_obj = datetime.strptime(date_str, "%Y%m%d")
    
    # Format the datetime object as a string in "DD/MM/YYYY" format
    formatted_date = date_obj.strftime("%d/%m/%Y")
    
    return formatted_date

## CORRECT ALL INVALID EMAILS

In [181]:
def replace_invalid_email(entry):
    if isinstance(entry, str) and 'Invalid.mail' in entry:
        return np.nan
    else:
        return entry

## CORRECT ALL INVALID PHONE NUMBERS

In [182]:
def replace_invalid_phone_number(entry):
    if isinstance(entry, str) and '@' in entry:
        return np.nan
    else:
        return entry

## CORRECT ALL CONTACT EQUAL TO 9999999999

In [183]:
def contact(entry):
    if isinstance(entry,str) and "9999999" in entry:
        return np.nan
    else:
        return entry

## ESTABLISH FEA AND FCA ACCOUNTS

In [184]:
def type_of_foreign_account(entry):
    if isinstance(entry,str) and "FEA" in entry:
        return "Foreign Exchange Account"
    elif isinstance(entry,str) and "FCA" in entry:
        return "Foreign Currency Account"
    else:
        return "N/A"

## MONTHLY ACTIVE DATA REPORT

In [185]:
def monthly_active():
    
    # Read template into pandas
    monthly_active_data = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Monthly Active Template.csv", dtype = "object", low_memory = False)
    
    # Populate each column
    monthly_active_data["Account Number"] = exclusion_list["CUSTOMER_ACCOUNT_NUMBER"]
    
    # Define new complete dataset that will be used to populate the template
#     dummy_monthly_active_data = pd.merge(monthly_active_data["Account Number"],LD, left_on = "Account Number", right_on = "CURRENT_ACCOUNT_NO", how = "left" )
#     dummy_monthly_active_data = pd.merge(dummy_monthly_active_data,exclusion_list, left_on = "Account Number", right_on = "CUSTOMER_ACCOUNT_NUMBER", how = "left")    

    # Populate each field with their respective requirements    
    monthly_active_data["Principal Amount (GHS Equivalent)"] = exclusion_list["PRINCIPAL_AMOUNT"]
    monthly_active_data["Interest Rate (% P.A.)"] = exclusion_list["INTEREST_RATE"]
    monthly_active_data["Interest Payable"] = exclusion_list["INT_AMOUNT_DUE"]
    monthly_active_data["Interest Paid"] = exclusion_list["INT_AMOUNT_PAID"]
    monthly_active_data["Status of Account"] = exclusion_list["STATUS_OF_ACCOUNT"]
    monthly_active_data["Status of Account"] = monthly_active_data["Status of Account"].str.title()
    
    # Filter for only active accounts
    monthly_active_data = monthly_active_data[monthly_active_data["Status of Account"] == "Active"]
    
    # Export dataset to csv
    monthly_active_data.to_csv(r"C:\Users\KATAKOF\Maymonthly_active.csv", index = False) 
    
    

In [186]:
# monthly = monthly_active()

## MONTHLY INDIVIDUAL DEPOSITOR REPORT

In [187]:
def individual():
    
    ## Read Monthly Individual Depositor Template
    individual_dep = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Monthly Individual Depositor Template.csv", dtype = "object", low_memory = False)
    
    ## Extract the individual accounts in the exclusion list
    individual_exclusion = exclusion_list[(exclusion_list["COMPANY_NAME"].isna()) & (~exclusion_list["DATE_OF_BIRTH"].isna())]
    
    ## Drop duplicates
    individual_dep_worksheet = individual_exclusion.drop_duplicates(subset = "BANK_SPECIFIC" , keep = "first")
    
    ## Assign the various columns with their right entries
    individual_dep["Customer ID"] = individual_dep_worksheet["BANK_SPECIFIC"]
    individual_dep["First Name"] = individual_dep_worksheet["FIRST_NAME"]
    individual_dep["Surname"] = individual_dep_worksheet["SURNAME"]
    individual_dep["Date of Birth"] = individual_dep_worksheet["DATE_OF_BIRTH"]
    individual_dep["Gender"] = individual_dep_worksheet["GENDER"]
    individual_dep["Email"] = individual_dep_worksheet["E_MAIL"]
    individual_dep["Mobile Phone Number"] = individual_dep_worksheet["MOBILE_PHONE_NUMBER"]
    individual_dep["Mobile Money Number"] = individual_dep_worksheet["MOBILE_MONEY_NUMBER"]
    individual_dep["Postal Address"] = individual_dep_worksheet["POSTAL_ADDRESS"]
    individual_dep["Date of Birth"] = individual_dep["Date of Birth"].apply(transform_date)
    individual_dep[["Mobile Phone Number","Mobile Money Number"]] = individual_dep[["Mobile Phone Number","Mobile Money Number"]].apply(replace_invalid_phone_number)
    individual_dep['Email'] = individual_dep['Email'].apply(replace_invalid_email)
    individual_dep["Residency"] = individual_dep_worksheet["COUNTRY"].str.replace("GH", "Ghana")
    
    
    ## Export final dataset
    individual_dep.to_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Report_exports\Mayindividual.csv", index = False)   
    

In [188]:
individual_depositor_sep = individual()

## MONTHLY CORPORATE DEPOSITOR REPORT

In [189]:
def corporate():
    
    ## Read Monthly Corporate Depositor Template
    corporate_dep = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Monthly Corporate Depositor Template.csv", dtype = "object", low_memory = False)
    
    ## Extract the Corporate Accounts in the Exclusion list
    corporate_exclusion_1 = exclusion_list[(~exclusion_list["COMPANY_NAME"].isna()) & (exclusion_list["DATE_OF_BIRTH"].isna())].drop_duplicates(subset = "BANK_SPECIFIC" , keep = "first")
    corporate_exclusion_2 = exclusion_list[(exclusion_list["COMPANY_NAME"].isna()) & (exclusion_list["DATE_OF_BIRTH"].isna())].drop_duplicates(subset = "BANK_SPECIFIC" , keep = "first")
    corporate_dep_worksheet = pd.concat([corporate_exclusion_1,corporate_exclusion_2])
    
    ## Assign the various columns with their right entries
    corporate_dep["Customer ID"] = corporate_dep_worksheet["BANK_SPECIFIC"]
    corporate_dep["Company Name"] = corporate_dep_worksheet["COMPANY_NAME"]
    corporate_dep["Contact"] = corporate_dep_worksheet["MAIN_PHONE_NUMBER"]
    corporate_dep["Contact"] = corporate_dep["Contact"].apply(contact)
    
    
    ## Replace those empty contact with that ID's corresponding "MOBILE_PHONE_NUMBER"
    corporate_dep["Contact"] = corporate_dep["Contact"].fillna(corporate_dep_worksheet["BANK_SPECIFIC"].map(corporate_dep_worksheet.set_index("BANK_SPECIFIC")["MOBILE_PHONE_NUMBER"]))
    corporate_dep["Contact"] = corporate_dep["Contact"].apply(contact)
    
    ## Export final dataset
    corporate_dep.to_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Report_exports\Maycorporate.csv", index = False)   
    
    
       

In [190]:
corporate_depositor_sep = corporate()

## MONTHLY DEPOSIT MASTER REPORT

In [191]:
def deposit_master():
    ## Import deposit master template
    dep_master = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Deposit Master Template.csv", dtype = "object", low_memory = False)
    
    ## Import account_ownership mapping and create new exclusion list
#     acc_ownership = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\ACC_OWNER_UNIT.csv", dtype = "object", low_memory = False)
#     exclusion = pd.merge(exclusion_list,acc_ownership, left_on = "ACCOUNT_BY_OWNERSHIP", right_on = "ACCOUNT_BY_OWNERSHIP" , how = "left")
    
    ## Assign deposit master columns to their respective entries
    exclusion_list_ = exclusion_list.drop_duplicates(subset = "CUSTOMER_ACCOUNT_NUMBER", keep = "first")
    dep_master[["Customer ID","Account Number","Currency","Type of Account"]] = exclusion_list_[["BANK_SPECIFIC","CUSTOMER_ACCOUNT_NUMBER","CURRENCY_OF_ACCOUNT","ACCOUNT_TYPE"]]
    dep_master["Product Name"] = exclusion_list_["PRODUCT_NAME"]
    dep_master["Status of Account"] = exclusion_list_["STATUS_OF_ACCOUNT"].str.title()
    dep_master["Account By Ownership"] = exclusion_list_["Bus_Unit"]
    dep_master["Type of Foreign Account"] = dep_master["Product Name"].apply(type_of_foreign_account)
    dep_master[["Principal Amount","Exchange Rate","Interest Payable","Interest Paid","Total Outstanding Amount"]] = exclusion_list_[["PRINCIPAL_AMOUNT","INTEREST_RATE","INT_AMOUNT_DUE","INT_AMOUNT_PAID","OUTSTANDING_AMOUNT"]]
    dep_master["Type of Foreign Account"] = dep_master["Product Name"].apply(type_of_foreign_account)
    dep_master["Interest Rate (% P.A.)"] = exclusion_list_["INTEREST_RATE"]
    
#     Export result into csv
    dep_master.to_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Report_exports\May_deposit master.csv", index = False)   
   
    return dep_master

In [192]:
monthly_deposit_master = deposit_master()

In [193]:
exclusion_list.columns

Index(['BANK_SPECIFIC', 'CUSTOMER_TYPE', 'TITLE', 'FIRST_NAME', 'MIDDLE_NAME',
       'SURNAME', 'PREVIOUS_NAME', 'COMPANY_NAME', 'GENDER', 'ID_TYPE',
       'ID_NUMBER', 'COMPANY_NUMBER', 'DATE_OF_BIRTH', 'HOME_ADDRESS',
       'POSTAL_ADDRESS', 'COUNTRY', 'E_MAIL', 'MAIN_PHONE_NUMBER',
       'MOBILE_PHONE_NUMBER', 'MOBILE_MONEY_NUMBER',
       'POLITICAL_EXPOSED_PERSON', 'ACCOUNT_TYPE', 'ACCOUNT_BY_OWNERSHIP',
       'CUSTOMER_ACCOUNT_NUMBER', 'PRODUCT_NAME', 'STATUS_OF_ACCOUNT',
       'EXCLUSION_TYPE', 'ACCOUNT_BRANCH', 'OWN_BALANCE_SHARE',
       'AUTH_NEGATIVE_BALANCE', 'CURRENCY_OF_ACCOUNT',
       'AMOUNT_BAL_IN_ORIG_CURRENCY', 'EXCHANGE_RATE',
       'ACCOUNT_BALANCE_IN_CEDIS', 'DEDUCT_OVER_DUE_LOAN', 'ACCOUNT_NO',
       'CUSTOMER_ID', 'INT_AMOUNT_DUE', 'INT_AMOUNT_PAID',
       'OUTSTANDING_AMOUNT', 'PRINCIPAL_AMOUNT', 'INTEREST_RATE',
       'CURRENT_ACCOUNT_NO', 'Bus_Unit'],
      dtype='object')

In [194]:
monthly_deposit_master.head()

,Customer ID,Account Number,Currency,Principal Amount,Exchange Rate,Principal Amount (GHS Equivalent),Type of Foreign Account,Type of Account,Account By Ownership,Date of Investment (if Applicable),...,Interest Paid,Total Outstanding Amount,Encumbered Amount,Status of Account,Exclusion Type,Product Name,Account Balance (Own Balance Share For Joint Accounts),Auth. Negative Balance,Overdue Loans,Sector
0,252041,2110252041015,GHS,NaN,NaN,NaN,N/A,Savings,Individual,NaN,...,NaN,NaN,NaN,Dormant,NaN,Savings-Personal,NaN,NaN,NaN,NaN
1,252590,2110252590017,GHS,NaN,NaN,NaN,N/A,Savings,Individual,NaN,...,NaN,NaN,NaN,Dormant,NaN,Savings-Personal,NaN,NaN,NaN,NaN
2,253192,2110253192013,GHS,NaN,NaN,NaN,N/A,Savings,Individual,NaN,...,NaN,NaN,NaN,Dormant,NaN,Savings-Personal,NaN,NaN,NaN,NaN
3,253192,1110253192017,GHS,NaN,NaN,NaN,N/A,Current Account,Individual,NaN,...,NaN,NaN,NaN,Dormant,NaN,Current-Personal,NaN,NaN,NaN,NaN
4,253057,2110253057029,GHS,NaN,NaN,NaN,N/A,Savings,Individual,NaN,...,NaN,NaN,NaN,Dormant,NaN,Savings-Personal,NaN,NaN,NaN,NaN


## MONTHLY INDIVIDUAL CUSTOMER DETAILS REPORT

In [195]:
# def ind_customer_details():
#     individual_cus = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Monthly Individual Customer Template.csv", dtype = "object", low_memory = False)

In [196]:
# individual_cus = pd.read_csv(r"C:\Users\KATAKOF\OneDrive - Universal Merchant Bank Limited\Documents\Compliance Reports\Templates\Monthly Individual Customer Template.csv", dtype = "object", low_memory = False)

In [197]:
# individual_cus.head()